Creating Simple txt to import to Qualtrics. [Link](https://www.qualtrics.com/support/survey-platform/survey-module/survey-tools/import-and-export-surveys/)

In [1]:
import pandas as pd
from collections import defaultdict
from collections import Counter
import os

In [2]:
df_messages = pd.read_csv('/home/pcallec/smoking_cessation_messages/results/vC/joined_output_vC_ChatGPT/joined_results_100.csv',
                          index_col=0)

In [3]:
df_messages.head()

,Message,model
0,You have the power to live your life without c...,gpt-j-6B
1,Your success isn’t determined by how much you ...,gpt-j-6B
2,"To stop smoking completely, you need to go thr...",gpt-j-6B
3,What are your interests? What do you like to d...,gpt-j-6B
4,"Talk to your friends, family, and colleagues a...",gpt-j-6B


In [4]:
l_models = df_messages["model"].unique()

In [5]:
l_models

array(['gpt-j-6B', 'opt-30b', 'original', 'ChatGPT', 'opt-13b',
       'opt-6.7b', 'bloom-7b1'], dtype=object)

In [6]:
path_directory = "/home/pcallec/smoking_cessation_messages/results/Qualtrics/messages_for_questionnaires"

n_questionnaires = 14

dict_count = defaultdict(int)
for i_questionnaire in range(n_questionnaires):
    
    df_temp = pd.DataFrame()
    
    # in order to distribute evenly the numbers of messages
    # among the questionnaires, we had 8 messages only two times in the 14 questionnaire
    # That is a model has: 12*7+2*8 = 100 messages in the 14 questionnaire
    
    # Quantity of messsage per model in each questionnaire
#            0 1 2 3 4 5 6 7 8 9 10 11 12 13
    # model1 8 7 7 7 7 7 7 8 7 7  7  7  7  7
    # model2 7 8 7 7 7 7 7 7 8 7  7  7  7  7
    # model3 7 7 8 7 7 7 7 7 7 8  7  7  7  7
    # model4 7 7 7 8 7 7 7 7 7 7  8  7  7  7
    # model5 7 7 7 7 8 7 7 7 7 7  7  8  7  7
    # model6 7 7 7 7 7 8 7 7 7 7  7  7  8  7
    # model7 7 7 7 7 7 7 8 7 7 7  7  7  7  8
    
    for i_model, model in enumerate(l_models):
        old_count = dict_count[model]
        if i_questionnaire % 7 == i_model:
            dict_count[model]+=8
        else:
            dict_count[model]+=7
    
        df_subset = df_messages[ df_messages["model"] == model].iloc[old_count:dict_count[model],:]
        df_temp = pd.concat([df_temp, df_subset], ignore_index=True)
#     print(i_questionnaire, Counter(df_temp["model"]))

    df_temp_shuffled = df_temp.sample(frac = 1, ignore_index=True)
    
    os.makedirs(path_directory, exist_ok = True)
    filename = f"messages_for_questionnaire_{i_questionnaire+1}.csv"
    path_file = os.path.join(path_directory, filename)
    
    df_temp_shuffled.to_csv(path_file)
#     break

# 1st round of questionnaires
First 7 questionnaires

In [7]:
path_output_directory = "/home/pcallec/smoking_cessation_messages/results/Qualtrics/questionnaires_code"
path_messages_directory = "/home/pcallec/smoking_cessation_messages/results/Qualtrics/messages_for_questionnaires"

n_questionnaires = 7

os.makedirs(path_output_directory, exist_ok = True)

for i_questionnaire in range(n_questionnaires):
    str_start_all = "[[AdvancedFormat]]\n\n"
    
    filename_messages = f"messages_for_questionnaire_{i_questionnaire+1}.csv"
    path_messages = os.path.join(path_messages_directory, filename_messages)
    
    df_messages = pd.read_csv(path_messages, index_col=0)

    str_all = str_start_all
    
    n_messages = df_messages.shape[0]
    
    str_block_intro = "[[Block: Message Intro]]\n\n"
    str_intro = f"[[Question:Text]]\n[[ID:Intro]]\n"+ \
                 "Creation of smoking cessation treatment messages can be time consuming. "+ \
                 "We have been working with computer scientists to develop methods that involve" + \
                 "using artificial intelligence to create new treatment messages. We would like " + \
                 "you to evaluate two sets of 50 messages each. The first set is attached to this " + \
                 "Qualtrics survey. Please use TTS standards and your best judgement and follow the " + \
                 "directions in the link to evaluate and improve the messages.\n\n"
    
    str_breakpage = "[[PageBreak]]\n\n"
    
    str_all = str_all + str_block_intro + str_intro + str_breakpage
    
    for i_message in range(n_messages):
        str_block_start = f"[[Block: Message {i_message+1}]]\n\n"
        message = df_messages.loc[i_message,"Message"]
        str_message_1 = f"[[Question:Text]]\n[[ID:M{i_message+1}_message_1]]" + \
                        f"\nMessage {i_message+1} out of {n_messages}:<br>\n" + \
                        f"{message}\n\n"
        str_question_style = f"[[Question:MC:SingleAnswer:Horizontal]]\n[[ID:M{i_message+1}_style]]\n"+ \
                              "Considering both content and style, how well-written is the message?\n"+ \
                              "[[Choices]]\n1-Poorly Written\n2\n3\n4\n5\n6\n7\n8\n9\n10-Very Well-Written\n\n"

        str_question_accuracy = f"[[Question:MC:SingleAnswer:Horizontal]]\n[[ID:M{i_message+1}_accuracy]]\n"+ \
                              "How accurate is this message? (Accurate refers to no misinformation and no factual errors)\n"+ \
                              "[[Choices]]\n1-Not at all\n2\n3\n4\n5\n6\n7\n8\n9\n10-Very Much\n\n"
       
        str_question_credibility = f"[[Question:MC:SingleAnswer:Horizontal]]\n[[ID:M{i_message+1}_credibility]]\n"+ \
                              "How credible does this message seem to you?\n"+ \
                              "[[Choices]]\n1-Not at all\n2\n3\n4\n5\n6\n7\n8\n9\n10-Very Much\n\n"

        str_question_persuasion = f"[[Question:MC:SingleAnswer:Horizontal]]\n[[ID:M{i_message+1}_persuasion]]\n"+ \
                              "To what extent do you feel this message can help smokers avoid smoking?\n"+\
                              "[[Choices]]\n1-Not at all\n2\n3\n4\n5\n6\n7\n8\n9\n10-Very Much\n\n"


        str_message_2 = f"[[Question:Text]]\n[[ID:M{i_message+1}_message_2]]" + \
                        f"\nMessage {i_message+1} out of {n_messages}:<br>\n" + \
                        f"{message}\n\n"
        str_question_revision_choice = f"[[Question:MC:SingleAnswer:Vertical]]\n[[ID:M{i_message+1}_revision_option]]\n"+\
                                        "Based on the TTS standards, do you feel this message needs further revision? "+\
                                        "If so, please revise it into a high-quality one in the text box.\n" +\
                                        "[[Choices]]\nNo need to revise, it meets TTS standards already\n"+\
                                        "Yes, it needs further revision. Please revise the message in the text box below.\n"
        
        str_question_revision_text = f"[[Question:TextEntry]]\n[[ID:M{i_message+1}_revision]]\n"+\
                                      "Revise here:\n\n"
#                               "[[SingleLine]]\n\n\n"
                
        str_question_template = f"[[Question:TextEntry]]\n[[ID:M{i_message+1}_timing]]\n\n"
        
        str_all = str_all + str_block_start + str_message_1 + str_question_style + str_question_accuracy  + \
                  str_question_credibility + str_question_persuasion + str_breakpage + str_message_2 + \
                  str_question_revision_choice + str_question_revision_text + str_question_template
    
#     print(i_questionnaire)
    filename = f"questionnaire_{i_questionnaire+1}.txt"
    path_file = os.path.join(path_output_directory, filename)
    f = open(path_file, "w", encoding='utf-8')
    f.write(str_all)
    f.close()

# 2nd round of messages

Next 7 questionnaires. Main difference is in the introduction

In [8]:
path_output_directory = "/home/pcallec/smoking_cessation_messages/results/Qualtrics/questionnaires_code"
path_messages_directory = "/home/pcallec/smoking_cessation_messages/results/Qualtrics/messages_for_questionnaires"

n_questionnaires = 14

os.makedirs(path_output_directory, exist_ok = True)

for i_questionnaire in range(7,n_questionnaires):
    str_start_all = "[[AdvancedFormat]]\n\n"
    
    filename_messages = f"messages_for_questionnaire_{i_questionnaire+1}.csv"
    path_messages = os.path.join(path_messages_directory, filename_messages)
    
    df_messages = pd.read_csv(path_messages, index_col=0)

    str_all = str_start_all
    
    n_messages = df_messages.shape[0]
    
    str_block_intro = "[[Block: Message Intro]]\n\n"
    str_intro = f"[[Question:Text]]\n[[ID:Intro]]\n"+ \
                 "Thank you for providing your feedback on the first set of 50 messages! "+ \
                 "We appreciate your effort and contribution to this project. " + \
                 "For the rest of the project, we would like you to review another 50 messages following " + \
                 "the same procedure. Please use TTS standards and your best judgment and " + \
                 "follow the directions in the link to evaluate and improve the messages. " + \
                 "Make sure to put the revised messages, instead of your comments in the text box. " + \
                 "DO NOT put comments in the revision box. Thank you!\n\n"
    
    str_breakpage = "[[PageBreak]]\n\n"
    
    str_all = str_all + str_block_intro + str_intro + str_breakpage
    
    for i_message in range(n_messages):
        str_block_start = f"[[Block: Message {i_message+1}]]\n\n"
        message = df_messages.loc[i_message,"Message"]
        str_message_1 = f"[[Question:Text]]\n[[ID:M{i_message+1}_message_1]]" + \
                        f"\nMessage {i_message+1} out of {n_messages}:<br>\n" + \
                        f"{message}\n\n"
        str_question_style = f"[[Question:MC:SingleAnswer:Horizontal]]\n[[ID:M{i_message+1}_style]]\n"+ \
                              "Considering both content and style, how well-written is the message?\n"+ \
                              "[[Choices]]\n1-Poorly Written\n2\n3\n4\n5\n6\n7\n8\n9\n10-Very Well-Written\n\n"

        str_question_accuracy = f"[[Question:MC:SingleAnswer:Horizontal]]\n[[ID:M{i_message+1}_accuracy]]\n"+ \
                              "How accurate is this message? (Accurate refers to no misinformation and no factual errors)\n"+ \
                              "[[Choices]]\n1-Not at all\n2\n3\n4\n5\n6\n7\n8\n9\n10-Very Much\n\n"
       
        str_question_credibility = f"[[Question:MC:SingleAnswer:Horizontal]]\n[[ID:M{i_message+1}_credibility]]\n"+ \
                              "How credible does this message seem to you?\n"+ \
                              "[[Choices]]\n1-Not at all\n2\n3\n4\n5\n6\n7\n8\n9\n10-Very Much\n\n"

        str_question_persuasion = f"[[Question:MC:SingleAnswer:Horizontal]]\n[[ID:M{i_message+1}_persuasion]]\n"+ \
                              "To what extent do you feel this message can help smokers avoid smoking?\n"+\
                              "[[Choices]]\n1-Not at all\n2\n3\n4\n5\n6\n7\n8\n9\n10-Very Much\n\n"


        str_message_2 = f"[[Question:Text]]\n[[ID:M{i_message+1}_message_2]]" + \
                        f"\nMessage {i_message+1} out of {n_messages}:<br>\n" + \
                        f"{message}\n\n"
        str_question_revision_choice = f"[[Question:MC:SingleAnswer:Vertical]]\n[[ID:M{i_message+1}_revision_option]]\n"+\
                                        "Based on the TTS standards, do you feel this message needs further revision? "+\
                                        "If so, please revise it into a high-quality one in the text box.\n" +\
                                        "[[Choices]]\nNo need to revise, it meets TTS standards already\n"+\
                                        "Yes, it needs further revision. Please revise the message in the text box below.\n"
        
        str_question_revision_text = f"[[Question:TextEntry]]\n[[ID:M{i_message+1}_revision]]\n"+\
                                      "Revise here:\n\n"
#                               "[[SingleLine]]\n\n\n"
                
        str_question_template = f"[[Question:TextEntry]]\n[[ID:M{i_message+1}_timing]]\n\n"
        
        str_all = str_all + str_block_start + str_message_1 + str_question_style + str_question_accuracy  + \
                  str_question_credibility + str_question_persuasion + str_breakpage + str_message_2 + \
                  str_question_revision_choice + str_question_revision_text + str_question_template
    
#     print(i_questionnaire)
    filename = f"questionnaire_{i_questionnaire+1}.txt"
    path_file = os.path.join(path_output_directory, filename)
    f = open(path_file, "w", encoding='utf-8')
    f.write(str_all)
    f.close()

In [9]:
df_messages.shape

(50, 2)